In [867]:
from statsmodels.tsa.stattools import adfuller
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
import json

In [868]:
data_df = pd.read_csv('./src/api/data/train_ML_IOT.csv', sep = ",", on_bad_lines='skip')

In [869]:
data_df

,DateTime,Junction,Vehicles,ID
0,2015-11-01 00:00:00,1,15,20151101001
1,2015-11-01 01:00:00,1,13,20151101011
2,2015-11-01 02:00:00,1,10,20151101021
3,2015-11-01 03:00:00,1,7,20151101031
4,2015-11-01 04:00:00,1,9,20151101041
...,...,...,...,...
48115,2017-06-30 19:00:00,4,11,20170630194
48116,2017-06-30 20:00:00,4,30,20170630204
48117,2017-06-30 21:00:00,4,16,20170630214
48118,2017-06-30 22:00:00,4,22,20170630224


In [870]:
grouped = data_df.groupby('DateTime')

In [871]:
grouped.head()

,DateTime,Junction,Vehicles,ID
0,2015-11-01 00:00:00,1,15,20151101001
1,2015-11-01 01:00:00,1,13,20151101011
2,2015-11-01 02:00:00,1,10,20151101021
3,2015-11-01 03:00:00,1,7,20151101031
4,2015-11-01 04:00:00,1,9,20151101041
...,...,...,...,...
48115,2017-06-30 19:00:00,4,11,20170630194
48116,2017-06-30 20:00:00,4,30,20170630204
48117,2017-06-30 21:00:00,4,16,20170630214
48118,2017-06-30 22:00:00,4,22,20170630224


In [872]:
data_df = data_df[data_df["Junction"] == 1]

In [873]:
data_df

,DateTime,Junction,Vehicles,ID
0,2015-11-01 00:00:00,1,15,20151101001
1,2015-11-01 01:00:00,1,13,20151101011
2,2015-11-01 02:00:00,1,10,20151101021
3,2015-11-01 03:00:00,1,7,20151101031
4,2015-11-01 04:00:00,1,9,20151101041
...,...,...,...,...
14587,2017-06-30 19:00:00,1,105,20170630191
14588,2017-06-30 20:00:00,1,96,20170630201
14589,2017-06-30 21:00:00,1,90,20170630211
14590,2017-06-30 22:00:00,1,84,20170630221


In [874]:
data_df.isna().sum()

DateTime    0
Junction    0
Vehicles    0
ID          0
dtype: int64

In [875]:
data_df["DateTime"] = pd.to_datetime(data_df["DateTime"])
d = data_df["DateTime"] 
data_df.index = pd.DatetimeIndex(d) #.sort_index(ascending=True, inplace=False)


/var/folders/xk/ct1qcm4j5ljdj441cx4jphvjp9l86x/T/ipykernel_63157/2952992933.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df["DateTime"] = pd.to_datetime(data_df["DateTime"])


In [876]:
data_df = data_df[:10000]

In [877]:
data_df

,DateTime,Junction,Vehicles,ID
DateTime,,,,
2015-11-01 00:00:00,2015-11-01 00:00:00,1,15,20151101001
2015-11-01 01:00:00,2015-11-01 01:00:00,1,13,20151101011
2015-11-01 02:00:00,2015-11-01 02:00:00,1,10,20151101021
2015-11-01 03:00:00,2015-11-01 03:00:00,1,7,20151101031
2015-11-01 04:00:00,2015-11-01 04:00:00,1,9,20151101041
...,...,...,...,...
2016-12-21 11:00:00,2016-12-21 11:00:00,1,76,20161221111
2016-12-21 12:00:00,2016-12-21 12:00:00,1,82,20161221121
2016-12-21 13:00:00,2016-12-21 13:00:00,1,76,20161221131


In [878]:
data_df.index = data_df.index + pd.DateOffset(years=3, days=17, months=1) - pd.DateOffset(hours=6)
data_df["DateTime"] = data_df.index

/var/folders/xk/ct1qcm4j5ljdj441cx4jphvjp9l86x/T/ipykernel_63157/1849961189.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df["DateTime"] = data_df.index


In [879]:
data_df = data_df.drop(columns=['Junction'])

In [880]:
data_df = data_df.drop(columns=['ID'])
data_df = data_df.rename(columns={"DateTime": "timestamp"})


In [881]:
data_df = data_df.rename(columns={"Vehicles": "vehicle count"})

In [882]:
data_df

,timestamp,vehicle count
DateTime,,
2018-12-17 18:00:00,2018-12-17 18:00:00,15
2018-12-17 19:00:00,2018-12-17 19:00:00,13
2018-12-17 20:00:00,2018-12-17 20:00:00,10
2018-12-17 21:00:00,2018-12-17 21:00:00,7
2018-12-17 22:00:00,2018-12-17 22:00:00,9
...,...,...
2020-02-07 05:00:00,2020-02-07 05:00:00,76
2020-02-07 06:00:00,2020-02-07 06:00:00,82
2020-02-07 07:00:00,2020-02-07 07:00:00,76


In [883]:
data_df["vehicle count"] = data_df["vehicle count"] * 8 + 200 + np.random.randint(1, 30, size=len(data_df)) 

In [884]:
(data_df["timestamp"].dt.hour > 23) & (data_df["timestamp"].dt.hour > 5)

DateTime
2018-12-17 18:00:00    False
2018-12-17 19:00:00    False
2018-12-17 20:00:00    False
2018-12-17 21:00:00    False
2018-12-17 22:00:00    False
                       ...  
2020-02-07 05:00:00    False
2020-02-07 06:00:00    False
2020-02-07 07:00:00    False
2020-02-07 08:00:00    False
2020-02-07 09:00:00    False
Name: timestamp, Length: 10000, dtype: bool

In [885]:
data_df["is_night"] = (data_df["timestamp"].dt.hour > 23) & (data_df["timestamp"].dt.hour < 5)
data_df["is_peak"] = ((data_df["timestamp"].dt.hour > 6) & (data_df["timestamp"].dt.hour < 10)) | ((data_df["timestamp"].dt.hour > 16) & (data_df["timestamp"].dt.hour < 20))

data_df["vehicle count"] = np.where(data_df["is_peak"], (data_df["vehicle count"] + 100), data_df["vehicle count"] - 200)
data_df["vehicle count"] = np.where(data_df["is_night"], (data_df["vehicle count"] / 4), data_df["vehicle count"])
data_df["vehicle count"] = np.where(data_df["timestamp"].dt.hour == 10, (data_df["vehicle count"] - 200), data_df["vehicle count"])


In [886]:
pd.to_datetime('2020-01-16T17:00')

Timestamp('2020-01-16 17:00:00')

In [887]:
data_df = pd.concat([data_df, 
                     pd.DataFrame({
    'vehicle count': [640], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-16 17:00:00')
}, index=[pd.to_datetime('2020-01-16 17:00:00')]),
                     pd.DataFrame({
    'vehicle count': [680], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-16 18:00:00')
}, index=[pd.to_datetime('2020-01-16 18:00')]),
pd.DataFrame({
    'vehicle count': [690], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-16 19:00:00')
}, index=[pd.to_datetime('2020-01-16 19:00')]),
pd.DataFrame({
    'vehicle count': [840], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-16 20:00:00')
}, index=[pd.to_datetime('2020-01-16 20:00')]),
pd.DataFrame({
    'vehicle count': [790], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-16 21:00:00')
}, index=[pd.to_datetime('2020-01-16 21:00')]),
pd.DataFrame({
    'vehicle count': [760], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-16 22:00:00')
}, index=[pd.to_datetime('2020-01-16 22:00')]),
pd.DataFrame({
    'vehicle count': [460], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-16 23:00:00')
}, index=[pd.to_datetime('2020-01-16 23:00')]),
pd.DataFrame({
    'vehicle count': [320], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 00:00:00')
}, index=[pd.to_datetime('2020-01-17 00:00')]),
pd.DataFrame({
    'vehicle count': [330], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 01:00:00')
}, index=[pd.to_datetime('2020-01-17 01:00')]),
pd.DataFrame({
    'vehicle count': [340], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 02:00:00')
}, index=[pd.to_datetime('2020-01-17 03:00')]),
pd.DataFrame({
    'vehicle count': [340], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 03:00:00')
}, index=[pd.to_datetime('2020-01-17 03:00')]),
pd.DataFrame({
    'vehicle count': [360], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 04:00:00')
  }, index=[pd.to_datetime('2020-01-17 04:00')]),
    pd.DataFrame({
    'vehicle count': [370], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 05:00:00')
  }, index=[pd.to_datetime('2020-01-17 05:00')]),
      pd.DataFrame({
    'vehicle count': [590], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 06:00:00')
  }, index=[pd.to_datetime('2020-01-17 06:00')]),
       pd.DataFrame({
    'vehicle count': [980], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 07:00:00')
  }, index=[pd.to_datetime('2020-01-17 07:00')]),
         pd.DataFrame({
    'vehicle count': [967], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 08:00:00')
  }, index=[pd.to_datetime('2020-01-17 08:00')]),
           pd.DataFrame({
    'vehicle count': [767], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 09:00:00')
  }, index=[pd.to_datetime('2020-01-17 09:00')]),
             pd.DataFrame({
    'vehicle count': [617], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 10:00:00')
  }, index=[pd.to_datetime('2020-01-17 10:00')]),
             pd.DataFrame({
    'vehicle count': [587], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 11:00:00')
  }, index=[pd.to_datetime('2020-01-17 11:00')]),
               pd.DataFrame({
    'vehicle count': [577], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 12:00:00')
  }, index=[pd.to_datetime('2020-01-17 12:00')]),
                 pd.DataFrame({
    'vehicle count': [687], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 13:00:00')
  }, index=[pd.to_datetime('2020-01-17 13:00')]),
                   pd.DataFrame({
    'vehicle count': [657], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 14:00:00')
  }, index=[pd.to_datetime('2020-01-17 14:00')]),
                   pd.DataFrame({
    'vehicle count': [657], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 15:00:00')
  }, index=[pd.to_datetime('2020-01-17 15:00')]),
                     pd.DataFrame({
    'vehicle count': [627], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 16:00:00')
  }, index=[pd.to_datetime('2020-01-17 16:00')]),
                       pd.DataFrame({
    'vehicle count': [857], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 17:00:00')
  }, index=[pd.to_datetime('2020-01-17 17:00')]),
                         pd.DataFrame({
    'vehicle count': [867], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 18:00:00')
  }, index=[pd.to_datetime('2020-01-17 18:00')])
])

In [888]:

data_df = pd.concat([data_df, 
                     pd.DataFrame({
    'vehicle count': [640], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-16 17:00:00')
}, index=[pd.to_datetime('2019-11-16 17:00:00')]),
                     pd.DataFrame({
    'vehicle count': [680], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-16 18:00:00')
}, index=[pd.to_datetime('2019-11-16 18:00')]),
pd.DataFrame({
    'vehicle count': [690], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-16 19:00:00')
}, index=[pd.to_datetime('2019-11-16 19:00')]),
pd.DataFrame({
    'vehicle count': [840], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-16 20:00:00')
}, index=[pd.to_datetime('2019-11-16 20:00')]),
pd.DataFrame({
    'vehicle count': [790], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-16 21:00:00')
}, index=[pd.to_datetime('2019-11-16 21:00')]),
pd.DataFrame({
    'vehicle count': [760], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-16 22:00:00')
}, index=[pd.to_datetime('2019-11-16 22:00')]),
pd.DataFrame({
    'vehicle count': [460], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-16 23:00:00')
}, index=[pd.to_datetime('2019-11-16 23:00')]),
pd.DataFrame({
    'vehicle count': [320], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-17 00:00:00')
}, index=[pd.to_datetime('2019-11-17 00:00')]),
pd.DataFrame({
    'vehicle count': [330], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-17 01:00:00')
}, index=[pd.to_datetime('2019-11-17 01:00')]),
pd.DataFrame({
    'vehicle count': [340], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-17 02:00:00')
}, index=[pd.to_datetime('2019-11-17 03:00')]),
pd.DataFrame({
    'vehicle count': [340], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-17 03:00:00')
}, index=[pd.to_datetime('2019-11-17 03:00')]),
pd.DataFrame({
    'vehicle count': [360], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-17 04:00:00')
  }, index=[pd.to_datetime('2019-11-17 04:00')]),
    pd.DataFrame({
    'vehicle count': [370], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-17 05:00:00')
  }, index=[pd.to_datetime('2019-11-17 05:00')]),
      pd.DataFrame({
    'vehicle count': [590], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-17 06:00:00')
  }, index=[pd.to_datetime('2019-11-17 06:00')]),
       pd.DataFrame({
    'vehicle count': [980], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-17 07:00:00')
  }, index=[pd.to_datetime('2019-11-17 07:00')]),
         pd.DataFrame({
    'vehicle count': [967], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-17 08:00:00')
  }, index=[pd.to_datetime('2019-11-17 08:00')]),
           pd.DataFrame({
    'vehicle count': [767], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-17 09:00:00')
  }, index=[pd.to_datetime('2019-11-17 09:00')]),
             pd.DataFrame({
    'vehicle count': [617], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-17 10:00:00')
  }, index=[pd.to_datetime('2019-11-17 10:00')]),
             pd.DataFrame({
    'vehicle count': [587], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-17 11:00:00')
  }, index=[pd.to_datetime('2019-11-17 11:00')]),
               pd.DataFrame({
    'vehicle count': [577], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-17 12:00:00')
  }, index=[pd.to_datetime('2019-11-17 12:00')]),
                 pd.DataFrame({
    'vehicle count': [687], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-17 13:00:00')
  }, index=[pd.to_datetime('2019-11-17 13:00')]),
                   pd.DataFrame({
    'vehicle count': [657], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-17 14:00:00')
  }, index=[pd.to_datetime('2019-11-17 14:00')]),
                   pd.DataFrame({
    'vehicle count': [657], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-17 15:00:00')
  }, index=[pd.to_datetime('2019-11-17 15:00')]),
                     pd.DataFrame({
    'vehicle count': [627], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-17 16:00:00')
  }, index=[pd.to_datetime('2019-11-17 16:00')]),
                       pd.DataFrame({
    'vehicle count': [857], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-17 17:00:00')
  }, index=[pd.to_datetime('2019-11-17 17:00')]),
                         pd.DataFrame({
    'vehicle count': [867], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2019-11-17 18:00:00')
  }, index=[pd.to_datetime('2019-11-17 18:00')]),
                            pd.DataFrame({
    'vehicle count': [340], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 01:00:00')
  }, index=[pd.to_datetime('2020-01-17 01:00:00')]),
                            pd.DataFrame({
    'vehicle count': [340], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 03:00:00')
  }, index=[pd.to_datetime('2020-01-17 03:00:00')]),
                          pd.DataFrame({
    'vehicle count': [340], "is_peak": False, "is_night": False, "timestamp": pd.to_datetime('2020-01-17 02:00:00')
  }, index=[pd.to_datetime('2020-01-17 02:00:00')])

  
])

In [889]:
data_df.loc[data_df.index == pd.to_datetime("2020-01-17 03:00:00"), "timestamp"] = pd.to_datetime("2020-01-17 03:00:00")

In [900]:
data_df[data_df.index >= pd.to_datetime("2020-01-17 00:00")].head(24)


,timestamp,vehicle count,is_night,is_peak
2020-01-17 00:00:00,2020-01-17 00:00:00,320.0,False,False
2020-01-17 01:00:00,2020-01-17 01:00:00,330.0,False,False
2020-01-17 02:00:00,2020-01-17 02:00:00,340.0,False,False
2020-01-17 03:00:00,2020-01-17 03:00:00,340.0,False,False
2020-01-17 04:00:00,2020-01-17 04:00:00,360.0,False,False
2020-01-17 05:00:00,2020-01-17 05:00:00,370.0,False,False
2020-01-17 06:00:00,2020-01-17 06:00:00,590.0,False,False
2020-01-17 07:00:00,2020-01-17 07:00:00,980.0,False,False
2020-01-17 08:00:00,2020-01-17 08:00:00,967.0,False,False
2020-01-17 09:00:00,2020-01-17 09:00:00,767.0,False,False


In [891]:
data_df = data_df[~(data_df.index.duplicated())]

In [892]:
data_df

,timestamp,vehicle count,is_night,is_peak
2018-12-17 18:00:00,2018-12-17 18:00:00,445.0,False,True
2018-12-17 19:00:00,2018-12-17 19:00:00,405.0,False,True
2018-12-17 20:00:00,2018-12-17 20:00:00,82.0,False,False
2018-12-17 21:00:00,2018-12-17 21:00:00,59.0,False,False
2018-12-17 22:00:00,2018-12-17 22:00:00,94.0,False,False
...,...,...,...,...
2019-11-17 14:00:00,2019-11-17 14:00:00,657.0,False,False
2019-11-17 15:00:00,2019-11-17 15:00:00,657.0,False,False
2019-11-17 16:00:00,2019-11-17 16:00:00,627.0,False,False
2019-11-17 17:00:00,2019-11-17 17:00:00,857.0,False,False


In [893]:
data_df = data_df.sort_index()

In [894]:
data_df["vehicle count"] = np.where(data_df["timestamp"].dt.hour == 10, (data_df["vehicle count"] + 200), data_df["vehicle count"])


In [895]:
data_df = data_df[(data_df.index >= pd.to_datetime('2019-12-04')) & (data_df.index < pd.to_datetime('2020-02-05'))]


In [896]:
data_df

,timestamp,vehicle count,is_night,is_peak
2019-12-04 00:00:00,2019-12-04 00:00:00,276.0,False,False
2019-12-04 01:00:00,2019-12-04 01:00:00,272.0,False,False
2019-12-04 02:00:00,2019-12-04 02:00:00,341.0,False,False
2019-12-04 03:00:00,2019-12-04 03:00:00,381.0,False,False
2019-12-04 04:00:00,2019-12-04 04:00:00,454.0,False,False
...,...,...,...,...
2020-02-04 19:00:00,2020-02-04 19:00:00,608.0,False,True
2020-02-04 20:00:00,2020-02-04 20:00:00,286.0,False,False
2020-02-04 21:00:00,2020-02-04 21:00:00,274.0,False,False
2020-02-04 22:00:00,2020-02-04 22:00:00,250.0,False,False


In [897]:
my_range_in1m = pd.date_range(start=data_df.index.min(), end=data_df.index.max(), freq='1h')


In [898]:
my_range_in1m[~my_range_in1m.isin(data_df.index)]


DatetimeIndex([], dtype='datetime64[ns]', freq='h')

In [899]:
data_df.to_json('./src/api/data/traffic_junction_simulated_15____09.json', orient="records")